# 파이썬으로 알리익스프레스 크롤링하기
---
최근 중국 직구 열풍이 일면서 알리익스프레스(AliExpress) 또는 테무(Temu) 등에서 물건을 찾으려고 하는 분들이 많습니다. 그런데, 두 웹사이트의 경우 제품 상세정보가 거의 중국어 또는 영어로 적혀있어서 우리가 구매를 위해서 확인하는 정보는 아주 적어지게 되지요. 그렇다면, 일단 제품명과 가격, URL 정도만 크롤링(crawling)을 해서 원하는 가격대의 물건을 1차적으로 가려내는 방향으로 접근을 해 보면 좋지 않을까요? 이번에 소개해 드릴 기술은 파이썬(Python)을 이용해서 알리익스프레스의 상품 정보를 크롤링하는 과정입니다. 이용한 패키지는 셀레니움(Selenium)입니다.
</p></br></br>

## 검색어 및 페이지 확인
---
이번에는 **공기청정기**를 검색한다는 상황을 가정해 코드를 작성해 보겠습니다. 알리의 검색결과는 `https://ko.aliexpress.com/w/wholesale-(keyword).html?page=(page)&g=y`와 같이구성되어 있으므로, `keyword` 부분에 검색어를, `page` 부분에 원하는 페이지 번호를 적으면 간편하지요. 맨 마지막 페이지 체크는 상품 정보가 나오지 않을 경우로 트리거를 설정해 두면 몇페이지까지 존재하는지 확인하는 과정을 생략할 수 있습니다. [# `last page check` 주석 참조](#제품_정보_수집하기)
</p></br></br>


In [ ]:
# import package
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# run webdriver
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
action = ActionChains(driver)

def check_price(t):
    try:
        return t.find_element(By.XPATH, './div[2]/div[3]/div[1]').text
    except:
        return t.find_element(By.XPATH, './div[2]/div[2]/div[1]').text  # AD item

keyword = '공기청정기'
page = 1

alli_res = pd.DataFrame()

</p></br></br>

## 자동 스크롤 설정
---
알리익스프레스에서 특정 제품을 검색하면, 스크롤을 내리기 전까지는 일정 개수의 제품만 로드되는 형태를 보입니다. 그래서, `selenium.webdriver.ActionChains` 기능을 이용해서 자동 스크롤을 구현해 보겠습니다. 동작 구현 방법은 아래와 같습니다.
</p></br></br>

1. 1차적으로 각 제품 정보를 수집한다.
2. 맨 마지막 물건이 나올 때까지 스크롤을 내린다.
3. 만약 맨 마지막 제품이 바뀐다면 새로운 상품이 로드되었다는 의미이므로 `1.`로 돌아간다.
4. 맨 마지막 제품이 바뀌지 않았다면 스크롤을 최대한 내렸다는 의미이므로 종료한다.
</p></br></br>

## 제품 정보 수집하기
---
알리익스프레스에서 제품 가격은 `//*[@id="card-list"]/div/div/a` 영역에 저장되어 있습니다. 해당 카드 영역에 제품 정보를 위치에 맞게 조합하는 방식이지요. 그래서 크롤링 프로그램에서는 앞서 설명드린 XPATH를 복사란 뒤 이름, 가격, URL 정보를 적절하게 뜯어내 보겠습니다. 각 정보는 아래와 같은 영역에 저장되어 있으니 참고해 주시고, 만약 스스로 위치를 알아보고 싶다면 크롬 기준 개발자 도구의 Select an Element 기능을 활용하시면 됩니다.
</p></br></br>

#### 각 카드 기준으로 제품 정보는 다음과 같은 위치에 저장되어 있습니다.
* 이름: `./div[2]/div[1]/h3`
* 가격: 일반 상품은 `./div[2]/div[3]/div[1]`에 저장되어 있고, 광고 상품은 `./div[2]/div[2]/div[1]` 영역에 저장되어 있습니다.
* URL: `a` 태그로 접속 링크가 구현되어 있으므로, 각 카드의 `href` attribute에 위치해 있습니다.
</p></br></br>

In [ ]:
while 1:
    # get page source
    url = f'https://ko.aliexpress.com/w/wholesale-{keyword}.html?page={page}&g=y'
    driver.get(url)
    last_elem = ''
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="card-list"]/div/div/a')))
    elem = driver.find_elements(By.XPATH, '//*[@id="card-list"]/div/div/a')  # item info.
    
    if elem == []: break  # last page check

    # scroll down
    while 1:
        action.scroll_to_element(elem[-1]).perform()
        elem = driver.find_elements(By.XPATH, '//*[@id="card-list"]/div/div/a')
        
        if elem[-1] == last_elem: break
        
        last_elem = elem[-1]

    product_name = [t.find_element(By.XPATH, './div[2]/div[1]/h3').text for t in elem]
    price = [check_price(t) for t in elem]
    url = [e.get_attribute('href') for e in elem]

    page += 1
    alli_res = pd.concat([alli_res, pd.DataFrame([product_name, price, url]).T])
    alli_res.to_excel('./alli_result.xlsx', index=False)

alli_res.columns = ['product name', 'price', 'url']
alli_res.to_excel('./alli_result.xlsx', index=False)

In [10]:
alli_res.head()

,product name,price,url
0,"냉장고 탈취제, 공기 청정기, 식품 보관 수명 연장제, 연기 제거, 애완 동물 변기...","₩1,300",https://ko.aliexpress.com/item/100500517369907...
1,"담배에서 간접 흡연을 걸러내는 다목적 재떨이, 공기 청정기 기능, 냄새 제거, 흡연...","₩2,000",https://www.aliexpress.com/gcp/300000512/KRupd...
2,"스마트 공기 청정기 음이온 발생기, H12 HEPA 필터 정화, 지능형 데스크탑 공...","₩16,505",https://ko.aliexpress.com/item/100500592833544...
3,"스마트 공기 청정기 음이온 발생기, H12 HEPA 필터 정화, 지능형 데스크탑 공...","₩16,546",https://ko.aliexpress.com/item/100500592834836...
4,"샤오미 미지아 스마트 항균 가습기 2, 가정용 UVC 99.9% 살균 공기 가습기,...","₩30,873",https://ko.aliexpress.com/item/100500573478742...
